In [2]:
from catboost import CatBoost,CatBoostRegressor,CatBoostClassifier, Pool, cv
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, accuracy_score
from catboost.datasets import msrank
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [3]:
# Load the data splits
train_data = pd.read_csv('../mcats/data/train_data.csv')
val_data = pd.read_csv('../mcats/data/test_data.csv')
test_data = pd.read_csv('../mcats/data/val_data.csv')

# Split the data into X and y
X_train = train_data.iloc[:, 1:-1]
y_train = train_data.iloc[:, -1]
X_val = val_data.iloc[:, 1:-1]
y_val = val_data.iloc[:, -1]
X_test = test_data.iloc[:, 1:-1]
y_test = test_data.iloc[:, -1]

In [16]:
# Split train data into two parts. First part - for baseline model,
# second part - for major model

splitted_data = train_test_split(X_train, y_train, test_size=0.5)
X_train_first, X_train_second, y_train_first, y_train_second = splitted_data

catboost_model = CatBoostRegressor(iterations=200, verbose=False)


# Prepare simple baselines (just mean target on first part of train pool).
baseline_value = y_train_first.mean()
train_baseline = np.array([baseline_value] * y_train_second.shape[0])
test_baseline = np.array([baseline_value] * y_test.shape[0])

# Create pools
train_pool = Pool(X_train_second, y_train_second, baseline=train_baseline)
test_pool = Pool(X_test, y_test, baseline=test_baseline)

# Train CatBoost model
catboost_model.fit(train_pool, eval_set=test_pool)

# Apply model on pool with baseline values
preds1 = catboost_model.predict(test_pool)

# Apply model on numpy.array and then add the baseline values
preds2 = test_baseline + catboost_model.predict(X_test)

# Check that preds have small diffs
assert (np.abs(preds1 - preds2) < 1e-6).all()

print(mean_squared_error(y_test, preds1))



##########################################
######### LINEAR BASELINE MODEL ##########
##########################################

# Train baseline model (linear regression) on first part of train pool
baseline_model = Ridge(alpha=3e3)
baseline_model.fit(X_train_first, y_train_first)

# Prepare baselines
train_baseline = baseline_model.predict(X_train_second)
test_baseline = baseline_model.predict(X_test)

# Create pools
train_pool = Pool(X_train_second, y_train_second, baseline=train_baseline)
test_pool = Pool(X_test, y_test, baseline=test_baseline)

# Train CatBoost model
catboost_model.fit(train_pool, eval_set=test_pool)

# Apply model on pool with baseline values
preds1 = catboost_model.predict(test_pool)

# Apply model on numpy.array and then add the baseline values
preds2 = baseline_model.predict(X_test) + catboost_model.predict(X_test)

# Check that preds have small diffs
assert (np.abs(preds1 - preds2) < 1e-6).all()

print(mean_squared_error(y_test, preds1))


3.4400216847534026
3.370889319672595


In [17]:
class AccuracyMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        best_class = np.argmax(approxes, axis=0)

        accuracy_sum = 0
        weight_sum = 0

        for i in range(len(target)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            accuracy_sum += w * (best_class[i] == target[i])

        return accuracy_sum, weight_sum

In [21]:
model = CatBoostClassifier(eval_metric=AccuracyMetric())
model

In [24]:
model = CatBoost()

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

randomized_search_result = model.randomized_search(grid,
                                                   X=X_train,
                                                   y=y_train,
                                                   plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 3.8828533	test: 3.9536562	best: 3.9536562 (0)	total: 32.7ms	remaining: 32.6s
1:	learn: 3.6332895	test: 3.6979009	best: 3.6979009 (1)	total: 37.9ms	remaining: 18.9s
2:	learn: 3.4108737	test: 3.4702767	best: 3.4702767 (2)	total: 42.2ms	remaining: 14s
3:	learn: 3.2176806	test: 3.2748041	best: 3.2748041 (3)	total: 48.3ms	remaining: 12s
4:	learn: 3.0534488	test: 3.1053506	best: 3.1053506 (4)	total: 53.3ms	remaining: 10.6s
5:	learn: 2.8982288	test: 2.9413732	best: 2.9413732 (5)	total: 61ms	remaining: 10.1s
6:	learn: 2.7667891	test: 2.8027068	best: 2.8027068 (6)	total: 66.7ms	remaining: 9.46s
7:	learn: 2.6520003	test: 2.6823295	best: 2.6823295 (7)	total: 73.1ms	remaining: 9.07s
8:	learn: 2.5517369	test: 2.5756230	best: 2.5756230 (8)	total: 79.5ms	remaining: 8.75s
9:	learn: 2.4720942	test: 2.4919854	best: 2.4919854 (9)	total: 84.9ms	remaining: 8.4s
10:	learn: 2.3943121	test: 2.4099401	best: 2.4099401 (10)	total: 89.8ms	remaining: 8.07s
11:	learn: 2.3291741	test: 2.3413100	best: 2.341

134:	learn: 1.3674243	test: 1.5487822	best: 1.5487822 (134)	total: 638ms	remaining: 4.08s
135:	learn: 1.3627036	test: 1.5451447	best: 1.5451447 (135)	total: 642ms	remaining: 4.08s
136:	learn: 1.3616012	test: 1.5444395	best: 1.5444395 (136)	total: 648ms	remaining: 4.08s
137:	learn: 1.3568122	test: 1.5414346	best: 1.5414346 (137)	total: 651ms	remaining: 4.07s
138:	learn: 1.3504115	test: 1.5392070	best: 1.5392070 (138)	total: 656ms	remaining: 4.06s
139:	learn: 1.3461161	test: 1.5372544	best: 1.5372544 (139)	total: 661ms	remaining: 4.06s
140:	learn: 1.3425696	test: 1.5362942	best: 1.5362942 (140)	total: 666ms	remaining: 4.05s
141:	learn: 1.3396634	test: 1.5332174	best: 1.5332174 (141)	total: 670ms	remaining: 4.05s
142:	learn: 1.3368097	test: 1.5318717	best: 1.5318717 (142)	total: 675ms	remaining: 4.04s
143:	learn: 1.3330164	test: 1.5298998	best: 1.5298998 (143)	total: 679ms	remaining: 4.04s
144:	learn: 1.3294083	test: 1.5269957	best: 1.5269957 (144)	total: 684ms	remaining: 4.03s
145:	learn

236:	learn: 1.0861043	test: 1.3992659	best: 1.3992659 (236)	total: 1.04s	remaining: 3.34s
237:	learn: 1.0834929	test: 1.3981338	best: 1.3981338 (237)	total: 1.04s	remaining: 3.34s
238:	learn: 1.0811752	test: 1.3972385	best: 1.3972385 (238)	total: 1.05s	remaining: 3.34s
239:	learn: 1.0787775	test: 1.3959924	best: 1.3959924 (239)	total: 1.05s	remaining: 3.33s
240:	learn: 1.0772322	test: 1.3953491	best: 1.3953491 (240)	total: 1.05s	remaining: 3.32s
241:	learn: 1.0758428	test: 1.3948217	best: 1.3948217 (241)	total: 1.06s	remaining: 3.32s
242:	learn: 1.0737144	test: 1.3929711	best: 1.3929711 (242)	total: 1.06s	remaining: 3.31s
243:	learn: 1.0729402	test: 1.3924411	best: 1.3924411 (243)	total: 1.06s	remaining: 3.3s
244:	learn: 1.0704201	test: 1.3916251	best: 1.3916251 (244)	total: 1.07s	remaining: 3.29s
245:	learn: 1.0682845	test: 1.3907773	best: 1.3907773 (245)	total: 1.07s	remaining: 3.28s
246:	learn: 1.0670790	test: 1.3901472	best: 1.3901472 (246)	total: 1.07s	remaining: 3.27s
247:	learn:

364:	learn: 0.8586827	test: 1.3013044	best: 1.3013044 (364)	total: 1.44s	remaining: 2.5s
365:	learn: 0.8569068	test: 1.2996258	best: 1.2996258 (365)	total: 1.44s	remaining: 2.49s
366:	learn: 0.8555901	test: 1.2992341	best: 1.2992341 (366)	total: 1.44s	remaining: 2.49s
367:	learn: 0.8539321	test: 1.2982570	best: 1.2982570 (367)	total: 1.44s	remaining: 2.48s
368:	learn: 0.8519050	test: 1.2971851	best: 1.2971851 (368)	total: 1.45s	remaining: 2.48s
369:	learn: 0.8501939	test: 1.2968878	best: 1.2968878 (369)	total: 1.45s	remaining: 2.47s
370:	learn: 0.8490790	test: 1.2958289	best: 1.2958289 (370)	total: 1.46s	remaining: 2.47s
371:	learn: 0.8475070	test: 1.2952670	best: 1.2952670 (371)	total: 1.46s	remaining: 2.47s
372:	learn: 0.8468813	test: 1.2950468	best: 1.2950468 (372)	total: 1.47s	remaining: 2.47s
373:	learn: 0.8450883	test: 1.2938288	best: 1.2938288 (373)	total: 1.47s	remaining: 2.46s
374:	learn: 0.8441935	test: 1.2930566	best: 1.2930566 (374)	total: 1.47s	remaining: 2.46s
375:	learn:

...

513:	learn: 0.3401060	test: 1.1837524	best: 1.1837524 (513)	total: 5.07s	remaining: 4.8s
514:	learn: 0.3386415	test: 1.1832642	best: 1.1832642 (514)	total: 5.09s	remaining: 4.79s
515:	learn: 0.3374842	test: 1.1824618	best: 1.1824618 (515)	total: 5.1s	remaining: 4.78s
516:	learn: 0.3370875	test: 1.1824184	best: 1.1824184 (516)	total: 5.11s	remaining: 4.78s
517:	learn: 0.3355852	test: 1.1816016	best: 1.1816016 (517)	total: 5.12s	remaining: 4.77s
518:	learn: 0.3346126	test: 1.1810696	best: 1.1810696 (518)	total: 5.13s	remaining: 4.75s
519:	learn: 0.3341826	test: 1.1809561	best: 1.1809561 (519)	total: 5.14s	remaining: 4.75s
520:	learn: 0.3331861	test: 1.1808605	best: 1.1808605 (520)	total: 5.15s	remaining: 4.74s
521:	learn: 0.3321627	test: 1.1802727	best: 1.1802727 (521)	total: 5.16s	remaining: 4.73s
522:	learn: 0.3317190	test: 1.1800954	best: 1.1800954 (522)	total: 5.17s	remaining: 4.72s
523:	learn: 0.3309055	test: 1.1800655	best: 1.1800655 (523)	total: 5.18s	remaining: 4.71s
524:	learn: 

619:	learn: 0.2588115	test: 1.1559614	best: 1.1559614 (619)	total: 6.11s	remaining: 3.74s
620:	learn: 0.2579776	test: 1.1556941	best: 1.1556941 (620)	total: 6.12s	remaining: 3.73s
621:	learn: 0.2574169	test: 1.1553852	best: 1.1553852 (621)	total: 6.13s	remaining: 3.72s
622:	learn: 0.2568423	test: 1.1552200	best: 1.1552200 (622)	total: 6.13s	remaining: 3.71s
623:	learn: 0.2560437	test: 1.1548572	best: 1.1548572 (623)	total: 6.15s	remaining: 3.7s
624:	learn: 0.2554690	test: 1.1544613	best: 1.1544613 (624)	total: 6.16s	remaining: 3.69s
625:	learn: 0.2549271	test: 1.1542916	best: 1.1542916 (625)	total: 6.17s	remaining: 3.68s
626:	learn: 0.2543309	test: 1.1541899	best: 1.1541899 (626)	total: 6.18s	remaining: 3.68s
627:	learn: 0.2537728	test: 1.1539794	best: 1.1539794 (627)	total: 6.19s	remaining: 3.67s
628:	learn: 0.2529632	test: 1.1536043	best: 1.1536043 (628)	total: 6.2s	remaining: 3.66s
629:	learn: 0.2522744	test: 1.1532901	best: 1.1532901 (629)	total: 6.21s	remaining: 3.65s
630:	learn: 

726:	learn: 0.2116958	test: 1.1408460	best: 1.1408460 (726)	total: 7.14s	remaining: 2.68s
727:	learn: 0.2112109	test: 1.1406222	best: 1.1406222 (727)	total: 7.15s	remaining: 2.67s
728:	learn: 0.2108413	test: 1.1405802	best: 1.1405802 (728)	total: 7.16s	remaining: 2.66s
729:	learn: 0.2108096	test: 1.1405687	best: 1.1405687 (729)	total: 7.17s	remaining: 2.65s
730:	learn: 0.2103186	test: 1.1404873	best: 1.1404873 (730)	total: 7.17s	remaining: 2.64s
731:	learn: 0.2097773	test: 1.1402930	best: 1.1402930 (731)	total: 7.18s	remaining: 2.63s
732:	learn: 0.2097436	test: 1.1403061	best: 1.1402930 (731)	total: 7.19s	remaining: 2.62s
733:	learn: 0.2092496	test: 1.1401651	best: 1.1401651 (733)	total: 7.2s	remaining: 2.61s
734:	learn: 0.2092449	test: 1.1401577	best: 1.1401577 (734)	total: 7.21s	remaining: 2.6s
735:	learn: 0.2090669	test: 1.1402155	best: 1.1401577 (734)	total: 7.22s	remaining: 2.59s
736:	learn: 0.2086937	test: 1.1401799	best: 1.1401577 (734)	total: 7.23s	remaining: 2.58s
737:	learn: 

831:	learn: 0.1710347	test: 1.1282029	best: 1.1282029 (831)	total: 8.17s	remaining: 1.65s
832:	learn: 0.1705709	test: 1.1281979	best: 1.1281979 (832)	total: 8.18s	remaining: 1.64s
833:	learn: 0.1702336	test: 1.1280611	best: 1.1280611 (833)	total: 8.19s	remaining: 1.63s
834:	learn: 0.1699925	test: 1.1280283	best: 1.1280283 (834)	total: 8.2s	remaining: 1.62s
835:	learn: 0.1695752	test: 1.1278861	best: 1.1278861 (835)	total: 8.21s	remaining: 1.61s
836:	learn: 0.1691104	test: 1.1280729	best: 1.1278861 (835)	total: 8.22s	remaining: 1.6s
837:	learn: 0.1687535	test: 1.1280207	best: 1.1278861 (835)	total: 8.23s	remaining: 1.59s
838:	learn: 0.1682033	test: 1.1278926	best: 1.1278861 (835)	total: 8.24s	remaining: 1.58s
839:	learn: 0.1676006	test: 1.1277150	best: 1.1277150 (839)	total: 8.25s	remaining: 1.57s
840:	learn: 0.1671825	test: 1.1276415	best: 1.1276415 (840)	total: 8.26s	remaining: 1.56s
841:	learn: 0.1669580	test: 1.1277036	best: 1.1276415 (840)	total: 8.27s	remaining: 1.55s
842:	learn: 

937:	learn: 0.1382645	test: 1.1201025	best: 1.1201025 (937)	total: 9.2s	remaining: 608ms
938:	learn: 0.1381227	test: 1.1200528	best: 1.1200528 (938)	total: 9.21s	remaining: 599ms
939:	learn: 0.1377354	test: 1.1199648	best: 1.1199648 (939)	total: 9.22s	remaining: 589ms
940:	learn: 0.1373987	test: 1.1196814	best: 1.1196814 (940)	total: 9.24s	remaining: 579ms
941:	learn: 0.1371897	test: 1.1196134	best: 1.1196134 (941)	total: 9.25s	remaining: 569ms
942:	learn: 0.1369954	test: 1.1196206	best: 1.1196134 (941)	total: 9.26s	remaining: 560ms
943:	learn: 0.1369427	test: 1.1195986	best: 1.1195986 (943)	total: 9.27s	remaining: 550ms
944:	learn: 0.1366953	test: 1.1196560	best: 1.1195986 (943)	total: 9.28s	remaining: 540ms
945:	learn: 0.1363739	test: 1.1196386	best: 1.1195986 (943)	total: 9.29s	remaining: 530ms
946:	learn: 0.1362962	test: 1.1196113	best: 1.1195986 (943)	total: 9.3s	remaining: 520ms
947:	learn: 0.1361380	test: 1.1196000	best: 1.1195986 (943)	total: 9.3s	remaining: 510ms
948:	learn: 0

In [4]:


# SEED = 2022
# def seed_everything(seed=SEED):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
    
# # seed_everything()

# RS_CV = 3
# RS_N_ITER = 10
# RS_N_JOBS = -1
# RS_SCORING = 'roc_auc'
# # search best parameter and model
# model = CatBoostClassifier()
# parameters = {'depth'         : sp_randInt(1, 10),
#               'learning_rate' : sp_randFloat(),
#               'iterations'    : sp_randInt(10, 100)
#              }

# rs = RandomizedSearchCV(estimator=model, 
#                            param_distributions = parameters,
#                            scoring=RS_SCORING,
#                            cv = RS_CV, 
#                            n_iter = RS_N_ITER, 
#                            n_jobs=RS_N_JOBS)
# rs.fit(X_train, y_train)

# # best model 
# print(rs.best_params_)
# bestModel = rs.best_estimator_

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [6]:

    model = CatBoostClassifier()
    parameters = {'depth'         : sp_randInt(4, 10),
                  'learning_rate' : sp_randFloat(),
                  'iterations'    : sp_randInt(10, 100)
                 }

    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)
    randm.fit(X_train, y_train)

    # Results from Random Search
    print("\n========================================================")
    print(" Results from Random Search " )
    print("========================================================")
    print("\n The best estimator across ALL searched params:\n",randm.best_estimator_)
    print("\n The best score across ALL searched params:\n",randm.best_score_)
    print("\n The best parameters across ALL searched params:\n",randm.best_params_)
    print("\n ========================================================")

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}